In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch
import torch.optim as optim
from torch.autograd import Variable

In [17]:
df = pd.read_csv('AUDUSD-2019-01.csv', header=None)

In [10]:
#df = df[2]

In [26]:
df.columns = ['Currency','Time','Bids','Asks']

In [76]:
returns = ((df.Bids - df.Bids.shift(1))/df.Bids.shift(1))[1:]

In [78]:
noisy_returns = returns + np.random.normal(0,0.000001,len(returns))

In [135]:
trainset = (noisy_returns[:int(len(noisy_returns)*0.7)].to_numpy(), returns[:int(len(noisy_returns)*0.7)].to_numpy())

In [136]:
testset = (noisy_returns[int(len(noisy_returns)*0.7):].to_numpy(), returns[int(len(noisy_returns)*0.7):].to_numpy())

In [137]:
batch_size=32

trainloader=DataLoader(trainset,batch_size=32,shuffle=True)
testloader=DataLoader(testset,batch_size=1,shuffle=True)

In [146]:
class denoising_model(nn.Module):
    def __init__(self):
        super(denoising_model,self).__init__()
        self.encoder=nn.Sequential(
                      nn.Linear(1,10),
                      nn.ReLU(True),
                      nn.Linear(10,10),
                      nn.ReLU(True),
                      nn.Linear(10,10),
                      nn.ReLU(True)

                      )

        self.decoder=nn.Sequential(
                      nn.Linear(10,10),
                      nn.ReLU(True),
                      nn.Linear(10,10),
                      nn.ReLU(True),
                      nn.Linear(10,1),
                      nn.Sigmoid(),
                      )
    
    def forward(self,x):
        x=self.encoder(x)
        x=self.decoder(x)

        return x

In [147]:
#We check whether cuda is available and choose device accordingly
if torch.cuda.is_available()==True:
    device="cuda:0"
else:
    device ="cpu"

model=denoising_model().to(device)
criterion=nn.MSELoss()
optimizer=optim.SGD(model.parameters(),lr=0.01,weight_decay=1e-5)


epochs=120
l=len(trainloader)
losslist=list()
epochloss=0
running_loss=0
for epoch in range(epochs):
  
    print("Entering Epoch: ",epoch)
    for dirty, clean in tqdm((trainloader)):

        dirty=dirty.view(dirty.size(0),-1).type(torch.FloatTensor)
        clean=clean.view(clean.size(0),-1).type(torch.FloatTensor)
        dirty,clean=dirty.to(device),clean.to(device)

        #-----------------Forward Pass----------------------
        output=model(dirty)
        loss=criterion(output,clean)
        #-----------------Backward Pass---------------------
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
        epochloss+=loss.item()
    #-----------------Log-------------------------------
    losslist.append(running_loss/l)
    running_loss=0
    print("======> epoch: {}/{}, Loss:{}".format(epoch,epochs,loss.item()))


Entering Epoch:  0





  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  1.04it/s]


======> epoch: 0/120, Loss:0.2229052037000656
Entering Epoch:  1





100%|██████████| 1/1 [00:00<00:00, 15.19it/s]


======> epoch: 1/120, Loss:0.2219453901052475
Entering Epoch:  2


...

======> epoch: 117/120, Loss:0.14547860622406006
Entering Epoch:  118





100%|██████████| 1/1 [00:00<00:00, 15.67it/s]


======> epoch: 118/120, Loss:0.14500229060649872
Entering Epoch:  119





100%|██████████| 1/1 [00:00<00:00, 14.53it/s]


======> epoch: 119/120, Loss:0.14452800154685974
